In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from api_keys import g_key
import time

In [17]:
file_path = "../WeatherPy/output_data/cities.csv"
cities_data = pd.read_csv(file_path)
cities_data.tail()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
540,Moose Jaw,50.40,-105.53,78.80,41,75,6.93,CA,1597687841
541,Tres Arroyos,-38.37,-60.28,57.06,41,0,15.88,AR,1597687841
542,Yulara,-25.24,130.99,57.20,36,0,6.93,AU,1597687842
543,Ferrol,43.48,-8.24,71.60,78,75,8.05,ES,1597687842
544,Biak,-0.91,122.88,74.77,72,100,8.41,ID,1597687842


In [67]:
gmaps.configure(api_key=g_key)
figure_layout = {
    'width': '400px',
    'height': '420px',
    'border': '1px solid black',
    'padding': '1px'
}
gmaps.figure(layout=figure_layout)
locations = cities_data[['Lat', 'Long']]
weight = cities_data['Humidity']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False)
heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 1
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [63]:
print(locations)

Lat    Long
73   31.67   38.66
90   -9.88  -56.09
134 -25.30  152.85
150 -16.23   39.91
157 -20.10  146.27
177  57.31   88.17
198 -13.28  -50.16
276  30.48 -115.95
319 -14.97   22.68
338 -25.17   45.05
341 -25.17   46.08
354  15.17   44.32
362  -9.66   20.39
379  30.85 -116.07
386  18.39   42.45
387  45.49   41.22
391  49.67   32.32
441 -11.03  -68.77
449 -18.51  -54.76
475  26.68  -80.67
482  18.49  -66.83
492  13.57   -8.03
509  56.54   89.30


In [4]:
columns = ['City', 'Lat', 'Long', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date']
city_choice = cities_data.loc[(cities_data["Max Temp"] > 70) & (cities_data["Max Temp"] < 80) & (
        cities_data["Wind Speed"] < 10) & (cities_data["Cloudiness"] == 0 ), columns]
city_choice.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
73,Turaif,31.67,38.66,77.00,44,0,3.36,SA,1597647492
90,Alta Floresta,-9.88,-56.09,70.29,40,0,1.34,BR,1597647494
134,Hervey Bay,-25.30,152.85,73.40,27,0,9.17,AU,1597647679
150,Angoche,-16.23,39.91,77.83,67,0,5.44,MZ,1597647683
157,Charters Towers,-20.10,146.27,76.19,19,0,3.76,AU,1597647684


In [27]:
hotel_df = city_choice
hotel_df['Hotel Name'] = ''
hotel_df['Hotel Rating'] = ''
hotel_df.head()
# len(hotel_df)

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Rating
73,Turaif,31.67,38.66,77.00,44,0,3.36,SA,1597647492,,
90,Alta Floresta,-9.88,-56.09,70.29,40,0,1.34,BR,1597647494,,
134,Hervey Bay,-25.30,152.85,73.40,27,0,9.17,AU,1597647679,,
150,Angoche,-16.23,39.91,77.83,67,0,5.44,MZ,1597647683,,
157,Charters Towers,-20.10,146.27,76.19,19,0,3.76,AU,1597647684,,


In [12]:
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
params = {
    'type': 'lodging',
    'radius': 5000,
    'key': g_key
}
resp = requests.get(url, params = params)
resp

<Response [200]>

In [37]:
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Long']
    location = f"{lat},{lng}"
    params['location'] = location
    print(f"Info: Retrieving results for hotels in city of: {row['City']} ({row['Country']}).")
    resp = requests.get(url, params = params)
    data = resp.json()
    results = data['results']
    try:
        print(f"Closest hotel is {results[0]['name']}. Guest rating is: {results[0]['rating']} stars")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Hotel Rating'] = results[0]['rating']
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = ('No Hotel Available')
        print(f'Missing field/result for hotel... skipping this one.')

Info: Retrieving results for hotels in city of: Turaif (SA).
Closest hotel is Swiss Spirit Hotel & Suites Turaif. Guest rating is: 3.5 stars
Info: Retrieving results for hotels in city of: Alta Floresta (BR).
Closest hotel is Floresta Amazonica Hotel. Guest rating is: 4.5 stars
Info: Retrieving results for hotels in city of: Hervey Bay (AU).
Closest hotel is Shelly Bay Resort. Guest rating is: 4.7 stars
Info: Retrieving results for hotels in city of: Angoche (MZ).
Closest hotel is Guest House HESADA. Guest rating is: 3.5 stars
Info: Retrieving results for hotels in city of: Charters Towers (AU).
Closest hotel is Cattleman's Rest Motor Inn. Guest rating is: 4 stars
Info: Retrieving results for hotels in city of: Tegul'det (RU).
Closest hotel is Gostinitsa Kedr. Guest rating is: 4.3 stars
Info: Retrieving results for hotels in city of: São Miguel do Araguaia (BR).
Closest hotel is Hotel Executivo Palace. Guest rating is: 4.2 stars
Info: Retrieving results for hotels in city of: San Quint

In [38]:
hotel_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Rating
73,Turaif,31.67,38.66,77.00,44,0,3.36,SA,1597647492,Swiss Spirit Hotel & Suites Turaif,3.5
90,Alta Floresta,-9.88,-56.09,70.29,40,0,1.34,BR,1597647494,Floresta Amazonica Hotel,4.5
134,Hervey Bay,-25.30,152.85,73.40,27,0,9.17,AU,1597647679,Shelly Bay Resort,4.7
150,Angoche,-16.23,39.91,77.83,67,0,5.44,MZ,1597647683,Guest House HESADA,3.5
157,Charters Towers,-20.10,146.27,76.19,19,0,3.76,AU,1597647684,Cattleman's Rest Motor Inn,4
177,Tegul'det,57.31,88.17,76.01,40,0,7.58,RU,1597647689,Gostinitsa Kedr,4.3
198,São Miguel do Araguaia,-13.28,-50.16,73.94,33,0,4.18,BR,1597647695,Hotel Executivo Palace,4.2
276,San Quintín,30.48,-115.95,73.26,72,0,3.38,MX,1597647714,Old Mill Hotel,4.4
319,Kalabo,-14.97,22.68,71.94,21,0,6.13,ZM,1597647724,Sikakubete Guest House Kalabo Town Council,3
338,Beloha,-25.17,45.05,76.12,50,0,6.31,MG,1597647727,No Hotel Available,


In [62]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]
# city = hotel_df['City']
# hotel_name = hotel_df['Hotel Name']
# hover_text = f"{city},{hotel_name}"
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))